In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from utils import config_parser
from rl import RL
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

2024-02-09 10:20:05.651208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-09 10:20:06.215769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

"""
Total sample num of fern: 20
16 = 4 + 4 + 4 + 4
    n0  n1  n2  n3
"""
args = Namespace(config='configs/fern.txt', expname='fern_test', basedir='./logs', datadir='../data/nerf_llff_data/fern', netdepth=8, netwidth=256, netdepth_fine=8, netwidth_fine=256, N_rand=1024, lrate=0.0005, lrate_decay=250, chunk=32768, netchunk=65536, no_batching=False, no_reload=False, ft_path=None, N_samples=64, N_importance=64, perturb=1.0, use_viewdirs=True, i_embed=0, multires=10, multires_views=4, raw_noise_std=1.0, render_only=False, render_test=False, render_factor=0, precrop_iters=0, precrop_frac=0.5, dataset_type='llff', testskip=8, shape='greek', white_bkgd=False, half_res=False, factor=8, no_ndc=False, lindisp=False, spherify=False, llffhold=8, i_print=500, i_img=500, i_weights=500, i_testset=500, i_video=500, n_iters=501, n=4, n0=4, baseline=True, using_sample_percentage=0.7, val_iter=100)


In [5]:
import os
import time
import numpy as np
from rl_models.dddqn import DDDQNAgent
from rl_env import NeRFENV

env = NeRFENV(args)

Loaded image data (378, 504, 3, 20) [378.         504.         407.56579161]
Loaded ../data/nerf_llff_data/fern 16.985296178676084 80.00209740336334
recentered (3, 5)
[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  1.4901161e-09]
 [ 0.0000000e+00  1.0000000e+00 -1.8730975e-09 -9.6857544e-09]
 [-0.0000000e+00  1.8730975e-09  1.0000000e+00  0.0000000e+00]]
Data:
(20, 3, 5) (20, 378, 504, 3) (20, 2)
HOLDOUT view is 12
Loaded llff (20, 378, 504, 3) (120, 3, 5) [378.     504.     407.5658] ../data/nerf_llff_data/fern
Auto LLFF holdout, 8
DEFINING BOUNDS
NEAR FAR 0.0 1.0
Found ckpts []
get rays
done, concats
shuffle rays
done
Begin
TRAIN views are [ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 17 18 19]
TEST views are [ 0  8 16]
VAL views are [ 0  8 16]
(20, 3, 5) (4845, 4)

RL Start==========
    Action Space: [[ 0  1  2  3]
 [ 0  1  2  4]
 [ 0  1  2  5]
 ...
 [15 16 18 19]
 [15 17 18 19]
 [16 17 18 19]]
    Episode Len: 5
              


In [6]:
agent = DDDQNAgent(lr=1e-3, gamma=0.99, n_actions=5, epsilon=1.0,
                batch_size=64, input_dims=[1], mem_size=100000)

In [2]:
np.random.choice([1,2,3,4], 2)

array([1, 4])

In [12]:
a = np.ones((1,378,504,3))
a.shape

(1, 378, 504, 3)

In [14]:
a.transpose(0,2,1,3).shape

(1, 504, 378, 3)